In [1]:
import os
import pickle

import autocnet
import pandas as pd

%pylab inline

root = '/work/users/cneubauer/ctx_trio'

Populating the interactive namespace from numpy and matplotlib


In [2]:
adjacency = {'G03_019262_1888_XN_08N203W.cal.cub':['P02_001949_1883_XN_08N203W.cal.cub', 'B19_017205_1888_XN_08N203W.cal.cub'],
             'P02_001949_1883_XN_08N203W.cal.cub':['B19_017205_1888_XN_08N203W.cal.cub', 'G03_019262_1888_XN_08N203W.cal.cub'],
             'B19_017205_1888_XN_08N203W.cal.cub':['P02_001949_1883_XN_08N203W.cal.cub','G03_019262_1888_XN_08N203W.cal.cub']}

In [3]:
# Create a candidate graph from the adjacency above
cg = autocnet.CandidateGraph.from_adjacency(adjacency, basepath=root)

In [4]:
# Load the features onto each node
# These are the raw extracted features that are 'interesting'
# This also loads the cameras that can be used for reprojection
for i, n in cg.nodes(data='data'):
    n.load_features(os.path.join(root, n['image_name'] + '_kps.h5'), format='hdf')
    with open(os.path.join(root, n['image_name'][:-8] + '.camera'), 'rb') as f:
        cam = pickle.load(f)
        n.camera = cam

In [ ]:
# cg.extract_features(extractor_parameters = {'nfeatures': 25})

In [5]:
cg.match()

In [6]:
cg.symmetry_checks()

In [7]:
cg.ratio_checks(clean_keys=['symmetry'])

In [8]:
cg.compute_fundamental_matrices(clean_keys=['ratio', 'symmetry'])

In [21]:
cg.edges[0, 1]['data']['fundamental_matrix']

array([[-2.84909036e-12, -4.84878457e-09,  1.20940875e-05],
       [ 4.87492124e-09, -1.94658839e-11, -2.91218730e-04],
       [-2.86055012e-05,  2.88872288e-04,  9.98741752e-01]])

In [18]:
numpy.save( '/work/users/cneubauer/ctx_trio/G03_P02_fundamental.npy' , cg.edges[0, 1]['data']['fundamental_matrix'] )

In [19]:
numpy.load( '/work/users/cneubauer/ctx_trio/G03_P02_fundamental.npy' )

array([[-2.84909036e-12, -4.84878457e-09,  1.20940875e-05],
       [ 4.87492124e-09, -1.94658839e-11, -2.91218730e-04],
       [-2.86055012e-05,  2.88872288e-04,  9.98741752e-01]])

In [22]:
cg.edges[0, 2]['data']['fundamental_matrix']

array([[ 1.12351323e-12,  3.46160710e-09, -4.16182753e-05],
       [-3.47727737e-09,  1.79627485e-11,  4.33839953e-04],
       [ 3.43068073e-05, -4.40168402e-04,  9.91844888e-01]])

In [25]:
numpy.save( '/work/users/cneubauer/ctx_trio/G03_B19_fundamental.npy' , cg.edges[0, 2]['data']['fundamental_matrix'] )

In [26]:
numpy.load( '/work/users/cneubauer/ctx_trio/G03_B19_fundamental.npy' )

array([[ 1.12351323e-12,  3.46160710e-09, -4.16182753e-05],
       [-3.47727737e-09,  1.79627485e-11,  4.33839953e-04],
       [ 3.43068073e-05, -4.40168402e-04,  9.91844888e-01]])

In [27]:
cg.edges[1, 2]['data']['fundamental_matrix']

array([[ 1.12099125e-12, -5.17322184e-09,  4.88214149e-05],
       [ 5.10244990e-09,  6.54444348e-12,  2.01692691e-04],
       [-1.99469902e-05, -1.89708602e-04,  9.99999936e-01]])

In [32]:
numpy.save( '/work/users/cneubauer/ctx_trio/P02_B19_fundamental.npy' , cg.edges[1, 2]['data']['fundamental_matrix'] )

In [33]:
numpy.load( '/work/users/cneubauer/ctx_trio/P02_B19_fundamental.npy' )

array([[ 1.12099125e-12, -5.17322184e-09,  4.88214149e-05],
       [ 5.10244990e-09,  6.54444348e-12,  2.01692691e-04],
       [-1.99469902e-05, -1.89708602e-04,  9.99999936e-01]])

In [29]:
cg.edges[1,2]['data'].destination


        NodeID: 2
        Image Name: B19_017205_1888_XN_08N203W.cal.cub
        Image PATH: /work/users/cneubauer/ctx_trio/B19_017205_1888_XN_08N203W.cal.cub
        Number Keypoints: 265448
        Available Masks : Empty DataFrame
Columns: []
Index: []
        Type: <class 'autocnet.graph.node.Node'>
        

In [ ]:
# Load the matches
# Since these matches can from autocnet_server (and a larger project) we remap the source and destination columns to the
#  node ids in this project.
for s, d, e in cg.edges(data='data'):
    source_key = e.source['image_name'][:3]
    destin_key = e.destination['image_name'][:3]
    matches = pd.read_csv(os.path.join(root, '{}_{}_matches.csv'.format(source_key, destin_key)))
    matches['source'] = s
    matches['destination'] = d
    e.matches = matches

In [ ]:
cg.edges[0,1]['data'].plot()

In [ ]:
cg.edges[0, 1]['data'].get_keypoints('source', index=matches['source_idx'])

In [ ]:
# cg.compute_fundamental_matrices()

In [34]:
cg[0][1]['data'].matches

,source_image,source_idx,destination_image,destination_idx,distance
0,0.0,43582.0,1.0,0.0,331.226471
1,0.0,109676.0,1.0,0.0,358.027771
2,0.0,12554.0,1.0,1.0,284.614502
3,0.0,199383.0,1.0,1.0,299.750000
4,0.0,44000.0,1.0,2.0,330.679474
5,0.0,50398.0,1.0,2.0,368.874084
6,0.0,99437.0,1.0,3.0,338.570801
7,0.0,87447.0,1.0,3.0,359.841766
8,0.0,40869.0,1.0,4.0,325.313690
9,0.0,63197.0,1.0,4.0,327.660492


In [ ]:
cg[0][1]['data']